In [2]:
import re
import pickle
import pandas as pd

with open('list', 'rb') as f:
	l = pickle.load(f)

In [ ]:
import requests
from tqdm import tqdm

s = requests.Session()
for p in tqdm(l):
	with open(p, 'w') as f:
		f.write(s.get("https://www.felixcloutier.com/x86/" + p).text)

In [ ]:
for p in tqdm(l):
	with open(p, 'r+') as f:
		tmp = f.read()
		tmp = tmp[tmp.find('<table'):tmp.find('</table>') + 8]
		f.seek(0)
		f.write(tmp)
		f.truncate()

In [4]:
things = []
for file in l:
	with open(file, 'r') as f:
		df = pd.read_html(f.read())[0]
	# if 'Opcode/Instruction' in df.keys():
	# 	things.append(df)
	if 'Opcode**' in df.keys() and 'Instruction' in df.keys():
		thing = []
		for i in range(df.shape[0]):
			thing.append(str(df['Opcode**'][i]) + ' ' + str(df['Instruction'][i]))
		df.insert(0, 'Opcode/Instruction', thing, True)
		df.drop(['Opcode**', 'Instruction'], 1, inplace=True)
		things.append(df)
	if 'Opcode***' in df.keys() and 'Instruction' in df.keys():
		thing = []
		for i in range(df.shape[0]):
			thing.append(str(df['Opcode***'][i]) + ' ' + str(df['Instruction'][i]))
		df.insert(0, 'Opcode/Instruction', thing, True)
		df.drop(['Opcode***', 'Instruction'], 1, inplace=True)
		things.append(df)

In [6]:
for thing in things:
	for i in range(thing.shape[0] - 1, -1, -1):
		if thing['Opcode/Instruction'][i].startswith('VEX') or thing['Opcode/Instruction'][i].startswith('EVEX'):
			thing.drop(i, inplace=True)
			continue
		# remove 64-bit incompatible instructions (in column 2)
		if not thing.iloc[i][2].startswith('V'):
			thing.drop(i, inplace=True)
things

[                  Opcode/Instruction Op/En 64-Bit Mode Compat/Leg Mode  \
 0                  D0 /2 RCL r/m8, 1    M1       Valid           Valid   
 1           REX + D0 /2 RCL r/m8*, 1    M1       Valid            N.E.   
 2                 D2 /2 RCL r/m8, CL    MC       Valid           Valid   
 3          REX + D2 /2 RCL r/m8*, CL    MC       Valid            N.E.   
 4            C0 /2 ib RCL r/m8, imm8    MI       Valid           Valid   
 5     REX + C0 /2 ib RCL r/m8*, imm8    MI       Valid            N.E.   
 6                 D1 /2 RCL r/m16, 1    M1       Valid           Valid   
 7                D3 /2 RCL r/m16, CL    MC       Valid           Valid   
 8           C1 /2 ib RCL r/m16, imm8    MI       Valid           Valid   
 9                 D1 /2 RCL r/m32, 1    M1       Valid           Valid   
 10        REX.W + D1 /2 RCL r/m64, 1    M1       Valid            N.E.   
 11               D3 /2 RCL r/m32, CL    MC       Valid           Valid   
 12       REX.W + D3 /2 R

In [7]:
for i in range(len(things) - 1, -1, -1):
	if (things[i].shape[0] == 0):
		things.pop(i)
things

[                  Opcode/Instruction Op/En 64-Bit Mode Compat/Leg Mode  \
 0                  D0 /2 RCL r/m8, 1    M1       Valid           Valid   
 1           REX + D0 /2 RCL r/m8*, 1    M1       Valid            N.E.   
 2                 D2 /2 RCL r/m8, CL    MC       Valid           Valid   
 3          REX + D2 /2 RCL r/m8*, CL    MC       Valid            N.E.   
 4            C0 /2 ib RCL r/m8, imm8    MI       Valid           Valid   
 5     REX + C0 /2 ib RCL r/m8*, imm8    MI       Valid            N.E.   
 6                 D1 /2 RCL r/m16, 1    M1       Valid           Valid   
 7                D3 /2 RCL r/m16, CL    MC       Valid           Valid   
 8           C1 /2 ib RCL r/m16, imm8    MI       Valid           Valid   
 9                 D1 /2 RCL r/m32, 1    M1       Valid           Valid   
 10        REX.W + D1 /2 RCL r/m64, 1    M1       Valid            N.E.   
 11               D3 /2 RCL r/m32, CL    MC       Valid           Valid   
 12       REX.W + D3 /2 R

In [8]:
for thing in things:
	for i in range(thing.shape[1] - 1, 0, -1):
		thing.drop(thing.columns[-1], axis=1, inplace=True)
things[1]

,Opcode/Instruction
0,"D0 /4 SAL r/m8, 1"
1,"REX + D0 /4 SAL r/m8**, 1"
2,"D2 /4 SAL r/m8, CL"
3,"REX + D2 /4 SAL r/m8**, CL"
4,"C0 /4 ib SAL r/m8, imm8"
5,"REX + C0 /4 ib SAL r/m8**, imm8"
6,"D1 /4 SAL r/m16, 1"
7,"D3 /4 SAL r/m16, CL"
8,"C1 /4 ib SAL r/m16, imm8"
9,"D1 /4 SAL r/m32, 1"


In [8]:
things[24]

,Opcode/Instruction
0,"24 ib AND AL, imm8"
1,"25 iw AND AX, imm16"
2,"25 id AND EAX, imm32"
3,"REX.W + 25 id AND RAX, imm32"
4,"80 /4 ib AND r/m8, imm8"
5,"REX + 80 /4 ib AND r/m8*, imm8"
6,"81 /4 iw AND r/m16, imm16"
7,"81 /4 id AND r/m32, imm32"
8,"REX.W + 81 /4 id AND r/m64, imm32"
9,"83 /4 ib AND r/m16, imm8"


In [10]:
s = []
for thing in things:
	for entry in thing['Opcode/Instruction']:
		s.append(entry)
s

['D0 /2 RCL r/m8, 1',
 'REX + D0 /2 RCL r/m8*, 1',
 'D2 /2 RCL r/m8, CL',
 'REX + D2 /2 RCL r/m8*, CL',
 'C0 /2 ib RCL r/m8, imm8',
 'REX + C0 /2 ib RCL r/m8*, imm8',
 'D1 /2 RCL r/m16, 1',
 'D3 /2 RCL r/m16, CL',
 'C1 /2 ib RCL r/m16, imm8',
 'D1 /2 RCL r/m32, 1',
 'REX.W + D1 /2 RCL r/m64, 1',
 'D3 /2 RCL r/m32, CL',
 'REX.W + D3 /2 RCL r/m64, CL',
 'C1 /2 ib RCL r/m32, imm8',
 'REX.W + C1 /2 ib RCL r/m64, imm8',
 'D0 /3 RCR r/m8, 1',
 'REX + D0 /3 RCR r/m8*, 1',
 'D2 /3 RCR r/m8, CL',
 'REX + D2 /3 RCR r/m8*, CL',
 'C0 /3 ib RCR r/m8, imm8',
 'REX + C0 /3 ib RCR r/m8*, imm8',
 'D1 /3 RCR r/m16, 1',
 'D3 /3 RCR r/m16, CL',
 'C1 /3 ib RCR r/m16, imm8',
 'D1 /3 RCR r/m32, 1',
 'REX.W + D1 /3 RCR r/m64, 1',
 'D3 /3 RCR r/m32, CL',
 'REX.W + D3 /3 RCR r/m64, CL',
 'C1 /3 ib RCR r/m32, imm8',
 'REX.W + C1 /3 ib RCR r/m64, imm8',
 'D0 /0 ROL r/m8, 1',
 'REX + D0 /0 ROL r/m8*, 1',
 'D2 /0 ROL r/m8, CL',
 'REX + D2 /0 ROL r/m8*, CL',
 'C0 /0 ib ROL r/m8, imm8',
 'REX + C0 /0 ib ROL r/m8*, im

In [28]:
tmp = [str(line) for line in s]
for i, line in enumerate(s):
	a = line.split('                                                                                     ')
	a[1] = a[1].replace(' ', '')
	s[i] = a[0] + ' ' + a[1]
s

['adc al imm8 14',
 'adc ax imm16 15',
 'adc eax imm32 15',
 'adc rax imm32 w15',
 'adc r/m8 imm8 80/2',
 'adc r/m16 imm16 81/2',
 'adc r/m32 imm32 81/2',
 'adc r/m64 imm32 w81/2',
 'adc r/m16 imm8 83/2',
 'adc r/m32 imm8 83/2',
 'adc r/m64 imm8 w83/2',
 'adc r/m8 r8 10',
 'adc r/m16 r16 11',
 'adc r/m32 r32 11',
 'adc r/m64 r64 w11',
 'adc r8 r/m8 12',
 'adc r16 r/m16 13',
 'adc r32 r/m32 13',
 'adc r64 r/m64 w13',
 'adcx r32 r/m32 p660f38f6',
 'adcx r64 r/m64 p66w0f38f6',
 'add al imm8 04',
 'add ax imm16 05',
 'add eax imm32 05',
 'add rax imm32 w05',
 'add r/m8 imm8 80/0',
 'add r/m16 imm16 81/0',
 'add r/m32 imm32 81/0',
 'add r/m64 imm32 w81/0',
 'add r/m16 imm8 83/0',
 'add r/m32 imm8 83/0',
 'add r/m64 imm8 w83/0',
 'add r/m8 r8 00',
 'add r/m16 r16 01',
 'add r/m32 r32 01',
 'add r/m64 r64 w01',
 'add r8 r/m8 02',
 'add r16 r/m16 03',
 'add r32 r/m32 03',
 'add r64 r/m64 w03',
 'addpd xmm1 xmm2/m128 p660f58',
 'addps xmm1 xmm2/m128 0f58',
 'addsd xmm1 xmm2/m64 pf20f58',
 'adds

In [11]:
# s = tmp
[print(x) for x in s]

D0 /2 RCL r/m8, 1
REX + D0 /2 RCL r/m8*, 1
D2 /2 RCL r/m8, CL
REX + D2 /2 RCL r/m8*, CL
C0 /2 ib RCL r/m8, imm8
REX + C0 /2 ib RCL r/m8*, imm8
D1 /2 RCL r/m16, 1
D3 /2 RCL r/m16, CL
C1 /2 ib RCL r/m16, imm8
D1 /2 RCL r/m32, 1
REX.W + D1 /2 RCL r/m64, 1
D3 /2 RCL r/m32, CL
REX.W + D3 /2 RCL r/m64, CL
C1 /2 ib RCL r/m32, imm8
REX.W + C1 /2 ib RCL r/m64, imm8
D0 /3 RCR r/m8, 1
REX + D0 /3 RCR r/m8*, 1
D2 /3 RCR r/m8, CL
REX + D2 /3 RCR r/m8*, CL
C0 /3 ib RCR r/m8, imm8
REX + C0 /3 ib RCR r/m8*, imm8
D1 /3 RCR r/m16, 1
D3 /3 RCR r/m16, CL
C1 /3 ib RCR r/m16, imm8
D1 /3 RCR r/m32, 1
REX.W + D1 /3 RCR r/m64, 1
D3 /3 RCR r/m32, CL
REX.W + D3 /3 RCR r/m64, CL
C1 /3 ib RCR r/m32, imm8
REX.W + C1 /3 ib RCR r/m64, imm8
D0 /0 ROL r/m8, 1
REX + D0 /0 ROL r/m8*, 1
D2 /0 ROL r/m8, CL
REX + D2 /0 ROL r/m8*, CL
C0 /0 ib ROL r/m8, imm8
REX + C0 /0 ib ROL r/m8*, imm8
D1 /0 ROL r/m16, 1
D3 /0 ROL r/m16, CL
C1 /0 ib ROL r/m16, imm8
D1 /0 ROL r/m32, 1
REX.W + D1 /0 ROL r/m64, 1
D3 /0 ROL r/m32, CL
REX.W + D

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [34]:
new = [x for x in s]
s.sort()
prev = s[0].split(' ')[0]
for line in s:
	if line.split(' ')[0] != prev:
		print()
	prev = line.split(' ')[0]
	print(line)
s = new

adc 0B IB 14
adc 0D ID 15
adc 0Q ID w15
adc 0W IW 15
adc MB IB 80/2
adc MB RB 10
adc MD IB 83/2
adc MD ID 81/2
adc MD RD 11
adc MQ IB w83/2
adc MQ ID w81/2
adc MQ RQ w11
adc MW IB 83/2
adc MW IW 81/2
adc MW RW 11
adc RB MB 12
adc RD MD 13
adc RQ MQ w13
adc RW MW 13

adcx RD MD p660f38f6
adcx RQ MQ p66w0f38f6

add 0B IB 04
add 0D ID 05
add 0Q ID w05
add 0W IW 05
add MB IB 80/0
add MB RB 00
add MD IB 83/0
add MD ID 81/0
add MD RD 01
add MQ IB w83/0
add MQ ID w81/0
add MQ RQ w01
add MW IB 83/0
add MW IW 81/0
add MW RW 01
add RB MB 02
add RD MD 03
add RQ MQ w03
add RW MW 03

addpd RX MX p660f58

addps RX MX 0f58

addsd RX MQ pf20f58
addsd RX RX pf20f58

addss RX MD pf30f58
addss RX RX pf30f58

addsubpd RX MX p660fd0

addsubps RX MX pf20fd0

adox RD MD pf30f38f6
adox RQ MQ pf3w0f38f6

aesdec RX MX p660f38de

aesdeclast RX MX p660f38df

aesenc RX MX p660f38dc

aesenclast RX MX p660f38dd

aesimc RX MX p660f38db

aeskeygenassist RX MX IB p660f3adf

and 0B IB 24
and 0D ID 25
and 0Q ID w25
and 0